# Practical Exercise 3.01: Equity Optimal Portfolios

In [ ]:
# Libraries for the practical exercises 2.1 to 2.5
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import scipy.optimize as sco
import statsmodels.api as sm
from datetime import datetime


In [ ]:
def get_adj_close(symbols, start, end=None):
    data = yf.download(symbols, start=start, end=end, auto_adjust=False, actions=False, group_by='ticker',progress=False)
    adj_close = data.xs('Adj Close', level=1, axis=1)
    # Rearrange columns exactly according to ticker list
    adj_close = adj_close[symbols]
    return adj_close

# List of companies
symbols = ['AAPL', 'AMZN', 'META', 'GOOGL','MSFT','NVDA', 'TSLA']
start = '2014-01-01'

prices = get_adj_close(symbols, start)
prices=prices.round(2)
prices

In [ ]:
# Let’s graphic the share price performance

fig, axes = plt.subplots(nrows=len(symbols), ncols = 1, figsize = (10,20))
for k, v in enumerate(sorted(symbols),0):
    axes[0].set_title('Adj Close Price Trends')
    axes[k].plot(prices[v], c=np.random.random(3))
    axes[k].legend([v], loc = 0)


In [ ]:
# Daily returns are calculated with these prices, starting from the cells without data.

daily_returns = prices.pct_change().dropna()
daily_returns.columns = symbols

mean_daily_returns = daily_returns.mean(axis=0)
daily_volt=daily_returns.std(axis=0)

# The annualized return and risk are calculated for each security.

an_volt = daily_volt * np.sqrt(252)
mean_returns = mean_daily_returns * 252
print('Annualized returns\n')
print(mean_returns.round(3))
print('______________________')
print('Annualized volatilities\n')
print(an_volt.round(3))


In [ ]:
# Let’s calculate the correlation matrix
daily_returns.corr().round(2)

In [ ]:
# The covariance matrix is calculated
cov_matrix = daily_returns.cov()

# We provide a specific target level of risk (volatility)
target_risk = float(input("Please enter the target volatility (e.g., 0.25 for 25%): "))

# We construct the function to calculate portfolio's annualized performance (volatility and return)
def portfolio_annualized_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns * weights)
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns

# We construct the function to maximize portfolio return given a target risk
def max_return_for_given_risk(mean_returns, cov_matrix, target_risk):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)

    # Constraint: weights sum to 1 and portfolio volatility equals the target risk
    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_annualized_performance(x, mean_returns, cov_matrix)[0]- target_risk}, {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

    # Bounds: each weight is between 0 and 1
    bounds = tuple((0, 1) for asset in range(num_assets))

    # Initial guess for the weights (equal distribution)
    initial_weights = num_assets * [1. / num_assets, ]

    # Perform the optimization to maximize return subject to constraints
    result = sco.minimize(lambda x: -portfolio_annualized_performance(x, mean_returns, cov_matrix)[1],
                          initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

# We find the optimal portfolio for the given target risk
optimal_portfolio = max_return_for_given_risk(mean_returns, cov_matrix, target_risk)
optimal_volatility, optimal_return = portfolio_annualized_performance(optimal_portfolio.x, mean_returns, cov_matrix)

# The optimal portfolio results are displayed
optimal_allocation = pd.DataFrame(optimal_portfolio.x, index=prices.columns, columns=['allocation'])
optimal_allocation.allocation = [round(i * 100, 2) for i in optimal_allocation.allocation]
optimal_allocation = optimal_allocation.T


print("\nOptimal Portfolio for Target Volatility of {:.2%}\n".format(target_risk))
print("Annualized Return (%):", round(optimal_return * 100, 3))
print("Annualized Volatility (%):", round(optimal_volatility * 100, 3))
print("\nAllocation:")
optimal_allocation


# Practical Exercise 3.02: Efficient Frontier

In [ ]:
# We provide the number of portfolios to be generated
num_portfolios = float(input("Please enter the number of portfolios to be generated (e.g., 25000): "))
num_portfolios = int(num_portfolios)


In [ ]:
# We provide the risk free rate
risk_free_rate = float(input("Please enter the risk free rate (e.g., 0.01 for 1%): "))


In [ ]:
# We construct a Portfolio Performance Function

def portfolio_annualized_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns * weights)
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns

# We generate Random Portfolios
def generate_random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((3, num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(len(symbols))
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_annualized_performance(weights, mean_returns, cov_matrix)
        results[0, i] = portfolio_std_dev
        results[1, i] = portfolio_return
        results[2, i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record

# We construct a function to minimize for Efficient Frontier

def minimize_volatility(mean_returns, cov_matrix, target_return):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)

    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_annualized_performance(x, mean_returns, cov_matrix)[1] - target_return},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

    bounds = tuple((0, 1) for asset in range(num_assets))
    result = sco.minimize(lambda x: portfolio_annualized_performance(x, mean_returns, cov_matrix)[0],
                          num_assets * [1. / num_assets], method='SLSQP', bounds=bounds, constraints=constraints)

    return result

# We generate Portfolios and identify Key Portfolios
results, weights = generate_random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate)

# We identify the Maximum Sharpe Ratio Portfolio, which is the portfolio of maximum slope in the efficient frontier:

max_sharpe_idx = np.argmax(results[2])
sdp, rp = results[0, max_sharpe_idx], results[1, max_sharpe_idx]
sharpe_ratio = (rp - risk_free_rate) / sdp
max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx], index=symbols, columns=['allocation'])
max_sharpe_allocation.allocation = [round(i*100,2) for i in max_sharpe_allocation.allocation]

# We identify the Minimum Variance Portfolio, which is the first portfolio of the efficient frontier:

min_vol_idx = np.argmin(results[0])
sdp_min, rp_min = results[0, min_vol_idx], results[1, min_vol_idx]
min_vol_allocation = pd.DataFrame(weights[min_vol_idx], index=symbols, columns=['allocation'])
min_vol_allocation.allocation = [round(i*100,2) for i in min_vol_allocation.allocation]

# We calculate the Efficient Frontier (Starting from the MVP)

target_returns = np.linspace(rp_min, max(mean_returns), 100)
efficient_volatilities = []
efficient_returns = []
for target_return in target_returns:
    optimized_result = minimize_volatility(mean_returns, cov_matrix, target_return)
    efficient_volatilities.append(optimized_result.fun)
    efficient_returns.append(target_return)
efficient_volatilities = np.array(efficient_volatilities)
efficient_returns = np.array(efficient_returns)


# We display Portfolio Characteristics:

print("\033[1mMaximum Sharpe Ratio Portfolio Allocation\033[0m\n")
print("Annualized Return:", round(rp*100, 2))
print("Annualized Volatility:", round(sdp*100, 2))
print("Sharpe Ratio:", round(sharpe_ratio, 2))
print(max_sharpe_allocation.T)

print("\n\033[1m Minimum Variance Portfolio Allocation\033[0m\n")
print("Annualized Return:", round(rp_min*100, 2))
print("Annualized Volatility:", round(sdp_min*100, 2))
print(min_vol_allocation.T)
print("_________________________________________________________ \n")

# We plot the Efficient Frontier

plt.figure(figsize=(8, 6))
plt.grid(True)
plt.scatter(results[0, :], results[1, :], c=results[2, :], cmap='YlGnBu', marker='o', s=10, alpha=0.3)
plt.colorbar()
plt.plot(efficient_volatilities, efficient_returns, linestyle='-', color='black', linewidth=2, label='Efficient Frontier')
plt.scatter(sdp, rp, marker='o', color='r', s=100, label='Maximum Sharpe ratio Portfolio')
plt.scatter(sdp_min, rp_min, marker='o', color='b', s=100, label='Minimum Variance Portfolio')
plt.title('EFFICIENT FRONTIER')
plt.xlabel('annualized volatility')
plt.ylabel('annualized returns')
plt.legend(labelspacing=0.8)
plt.show()


# Practical Exercise 3.03: Capital Market Line

In [ ]:
# Additional Code for Incorporating CML

# We calculate portfolio return and weights in market portfolio and risk-free asset

cml_return = risk_free_rate + sharpe_ratio * target_risk
weight_market_portfolio = target_risk / sdp
weight_risk_free = 1 - weight_market_portfolio

# We calculate the portfolio characteristics for the target risk

target_portfolio_allocation = weight_market_portfolio * max_sharpe_allocation['allocation'] / 100


# We determine if it's a lending or borrowing portfolio

if weight_market_portfolio < 1:
    portfolio_type = "Lending Portfolio"
else:
    portfolio_type = "Borrowing Portfolio"


# We display the characteristics of the portfolio with the target risk

print(portfolio_type)
print("Target Risk (%):", round(target_risk*100, 2))
print("Return (%):", round(cml_return*100, 2))
print("Weight in Market Portfolio (%):", round(weight_market_portfolio*100, 2))
print("Weight in Risk-Free Asset (%):", round(weight_risk_free*100, 2))


In [ ]:
# We plot Efficient Frontier

plt.figure(figsize=(8, 6))
plt.grid(True)
plt.scatter(results[0, :], results[1, :], c=results[2, :], cmap='YlGnBu', marker='o', s=10, alpha=0.3)
plt.colorbar()
plt.plot(efficient_volatilities, efficient_returns, linestyle='-', color='black', linewidth=2, label='Efficient Frontier')
plt.scatter(sdp, rp, marker='o', color='r', s=100, label='Maximum Sharpe ratio Portfolio')
plt.scatter(sdp_min, rp_min, marker='o', color='b', s=100, label='Minimum Variance Portfolio')
plt.title('EFFICIENT FRONTIER')
plt.xlabel('annualized volatility')
plt.ylabel('annualized returns')
plt.legend(labelspacing=0.8)

# We plot CML

cml_x = np.linspace(0, max(results[0, :]) * 1.5, 100)
cml_y = risk_free_rate + sharpe_ratio * cml_x
plt.plot(cml_x, cml_y, linestyle='-', color='brown', label='Capital Market Line')

# Plot Tobin Portfolio at Target Risk

plt.scatter(target_risk, cml_return, marker='o', color='g', s=100, label=f'{portfolio_type}')

plt.title('CAPITAL MARKET LINE (CML)')
plt.xlabel('annualized volatility')
plt.ylabel('annualized returns')
plt.legend(labelspacing=0.8)
plt.show()


# Practical Exercise 3.04: Security Market Line

In [ ]:
# List of companies
symbols = ['AAPL', 'AMZN', 'META', 'GOOGL','MSFT','NVDA', 'TSLA']
start = '2014-01-01'
prices = get_adj_close(symbols, start)
symbol = ['^GSPC']
start = '2014-01-01'
index = get_adj_close(symbol, start)
index = index.rename(columns={'^GSPC': 'S&P500'})

In [ ]:
def get_portfolio_data(prices, index):
    # Combine share prices and index
    adj_close_data = prices.join(index, how='inner')

    # Daily returns
    portfolio_daily_returns = prices.pct_change(fill_method=None).dropna()
    market_daily_returns = index['S&P500'].pct_change(fill_method=None).dropna()

    # Average annual returns (252 trading days)
    portfolio_mean_returns = portfolio_daily_returns.mean() * 252
    market_mean_return = market_daily_returns.mean() * 252

    return adj_close_data, portfolio_daily_returns, market_daily_returns, portfolio_mean_returns, market_mean_return


In [ ]:
# Apply the function to the list of stocks

symbols = ['AAPL', 'AMZN', 'META', 'GOOGL', 'MSFT', 'NVDA', 'TSLA']
start = '2014-01-01'

prices = get_adj_close(symbols, start)
index = get_adj_close(['^GSPC'], start).rename(columns={'^GSPC': 'S&P500'})

adj_close_data, portfolio_daily_returns, market_daily_returns, portfolio_mean_returns, market_mean_return = \
    get_portfolio_data(prices, index)

print("\nMarket annualized return:", round(market_mean_return, 4))
print("\nStock annualized returns:")
print(portfolio_mean_returns.round(4))



In [ ]:
# We define the start and end dates for the period you want to analyze

days_before = float(input("Please enter the number of previous days (e.g., 50):"))
end_date=adj_close_data.index.max()
start_date = end_date - pd.DateOffset(days=days_before)
start_date = pd.to_datetime(start_date,format='%Y%m%d')
end_date = pd.to_datetime(end_date,format='%Y%m%d')
filtered_data = adj_close_data.loc[start_date:end_date]
num_observations = len(filtered_data)
print('Start date:', start_date)
print('End date:', end_date)
print("Trading sessions:", num_observations)



In [ ]:
# Beta is calculated on a specific period basis

# We filter the daily returns data for the specified date range

portfolio_filtered = portfolio_daily_returns.loc[start_date:end_date]
portfolio_filtered=portfolio_filtered.reindex(columns=symbols)
market_filtered = market_daily_returns.loc[start_date:end_date]

# We calculate the beta for each asset within the selected date range

X = market_filtered
y = portfolio_filtered

X1 = sm.add_constant(X)
model = sm.OLS(y, X1)
reg = model.fit()
beta = reg.params[1:]  # Extract the beta coefficients
beta.columns = (symbols)

# We display the calculated betas
print('Betas')
beta.round(2)


When you estimate a stock’s beta (its sensitivity to market movements), you do it by running a linear regression of the stock’s returns against market returns.
However, not all regressions give a strong relationship — sometimes the R² (coefficient of determination) is low, meaning the market doesn’t explain much of that stock’s behavior.

For unreliable betas, it adjusts the value toward the market beta (which is 1) using this formula:

Beta_modified=0.667×Beta_stock+0.333×Beta_market

This is a Blume-type adjustment, commonly used in finance to make extreme betas more realistic — high betas are adjusted down, and low betas are adjusted up.

In [ ]:
# Let's calculate R² for every stock and correct stock beta if R² < 0.5  following Blume-type adjustment ====================

# Calculate R² for each asset
r2_values = {}
for t in symbols:
    yi = y[t].dropna()
    yhat_i = reg.fittedvalues[t].reindex(yi.index)
    resid_i = (yi - yhat_i).dropna()

    yi_aligned = yi.reindex(resid_i.index)
    ssr = np.sum(resid_i**2)
    sst = np.sum((yi_aligned - yi_aligned.mean())**2)
    r2 = np.nan if sst == 0 else (1 - ssr / sst)
    r2_values[t] = r2

r2_series = pd.Series(r2_values).reindex(symbols)
print("\n\033[1mR² per asset:\033[0m\n")
print(r2_series)

# Apply modified beta only if R² < 0.5
w = 0.667
beta_market = 1.0

# Convert beta (DataFrame) to Series for easy operation
if isinstance(beta, pd.DataFrame):
    beta_series = beta.iloc[0].astype(float)
else:
    beta_series = pd.Series(beta, index=symbol, dtype=float)

# Create a copy for the final version
beta_final = beta_series.copy()

# Apply correction only to assets with R² < 0.5
for t in symbols:
    if r2_series[t] < 0.5:
        beta_final[t] = w * beta_series[t] + (1 - w) * beta_market

print("\n\033[1mFinal beta (correcting only those with R² < 0.5):\033[0m\n")
print(beta_final)

In [ ]:
# The required profitability of each security is defined:
# Ask for the risk free rate
risk_free_rate = float(input("Please enter the risk free rate (e.g., 0.01 for 1%): "))
CAPMreturn=risk_free_rate+beta_final.T*(market_mean_return-risk_free_rate)
CAPMreturn=CAPMreturn.T
print('\n')
print('\033[1mRequired returns:\033[0m\n')
print (CAPMreturn)


In [ ]:
# Transpose both dataframes to make tickers rows (indexes)
Expected_Returns = portfolio_mean_returns.T
Required_Returns = CAPMreturn.T

# If there is an extra index (such as ^GSPC), we remove it.
Required_Returns = Required_Returns.reset_index(drop=True)
Required_Returns.index = Expected_Returns.index

# Now we join both dataframes by the index (the tickers)
combined_df = pd.concat([Expected_Returns*100, Required_Returns*100], axis=1)

# We rename the columns for clarity
combined_df.columns = ['Expected return (%)', 'Required return (%)']

# Add a new column to indicate whether it is undervalued, overvalued or fairly valued.
combined_df['valuation'] = combined_df.apply(
    lambda row: 'Undervalued' if row['Required return (%)'] < row['Expected return (%)'] else
                'Overvalued' if row['Required return (%)'] > row['Expected return (%)'] else
                'Fairly Valued', axis=1)

# Display the combined dataframe
combined_df.round(2)


In [ ]:
# The function that reflects the performance of the portfolios is defined:

def portfolio_annualized_performance(weights, CAPMreturn, beta_final):
    returnsp = np.sum(CAPMreturn*weights )
    betap = np.sum(beta_final*weights)
    return betap, returnsp

# The number of portfolios to be created is determined:
num_portfolios = int(input("Please enter the number of portfolios to simulate (e.g., 25000 for 25,000):"))


In [ ]:
# Portfolios are randomly created by attributing their profitability and risk:

def random_portfolios(num_portfolios, CAPMreturn, beta_final):
    results = np.zeros((3,num_portfolios))
    weights_list = []
    for i in range(num_portfolios):
        weights = np.random.random(len(symbols))
        weights /= np.sum(weights)
        weights_list.append(weights)
        portfolio_beta, portfolio_return = portfolio_annualized_performance(weights, CAPMreturn, beta_final)
        results[0,i] = portfolio_beta
        results[1,i] = portfolio_return

    return results, weights_list
# The results tables of betas and returns are converted into matrices:

CAPMreturn_array=np.array(CAPMreturn)
beta_array=np.array(beta_final)
portfolio_mean_returns_array=np.array(portfolio_mean_returns)

# The role of the SML is defined on the basis of the portfolios created, incorporating the individual securities:

def Simulated_SML(CAPMreturn_array, beta_array, num_portfolios):
    results, weights = random_portfolios(num_portfolios,CAPMreturn_array, beta_array)
    betas = [x/10 for x in range(30)]
    assetReturns =[risk_free_rate+x*(market_mean_return-risk_free_rate) for x in betas]

    plt.figure(figsize=(11, 6))
    plt.grid(True)
    plt.scatter(results[0,:],results [1,:], marker='o', s=10, alpha=0.3)
    plt.scatter(beta_array,CAPMreturn_array,marker='o',color='r',s=100,edgecolors='k')
    plt.scatter(beta,portfolio_mean_returns_array,marker='o', color='y', s=100,edgecolors='k')
    plt.scatter(1,market_mean_return,marker='o',color='b',s=100, label='Market Index',edgecolors='k')
    plt.scatter(0, risk_free_rate, marker='o',color='k',s=100, label='Risk free asset')
    plt.plot(betas,assetReturns, label='SML')
    plt.legend(loc='center left', bbox_to_anchor=(-0.3, 0.5), frameon=False)

     # Annotate each ticker on the plot
    for i in range(len(symbols)):
        plt.annotate(symbols[i], (beta_array.T[i], CAPMreturn_array.T[i]), xytext=(5,5), textcoords='offset points', fontsize=6, color='black')
        plt.annotate(symbols[i], (beta_array.T[i], portfolio_mean_returns_array.T[i]), xytext=(5,5), textcoords='offset points', fontsize=8, color='black')

    plt.title('Simulated Security Market Line')
    plt.xlabel('beta')
    plt.ylabel('returns (%)')
    plt.legend(labelspacing=0.8)
    plt.show()

# Results are shown

Simulated_SML(CAPMreturn_array, beta_array, num_portfolios)


# Practical Exercise 3.05: CAPM Portfolio

In [ ]:
!pip install pulp

In [ ]:
import pulp as pl
# It is a Python library used for modeling and
# solving optimization problems using linear programming
sorted(symbols)


In [ ]:
# The betas are needed in dictionary format for the optimization function.
# To do this: # First, the betas are passed to matrix format,
# then to list format, and finally to dictionary format:
beta_final=beta_final.round(2)
beta2=np.array(beta_final)
beta2=beta2.T
beta2=beta2.tolist()
betastocks=[]
for i in range (len(symbols)):
    betastocks.append(beta2[i])
betastocks=dict(zip(sorted(symbols),betastocks))

# In order to set the target beta, it is necessary to know
# the range of betas of the securities that make up the portfolio:
print ("\033[1mRange of Betas\033[0m")
print ("Maximum Beta=",max(beta2))
print ("Minimum Beta=",min(beta2))


In [ ]:
# The beta that fits the investor's profile is established:
BetaTarget = float(input("Please enter the beta seeked (e.g., 1.5):"))


In [ ]:
#This code is setting up a linear programming problem for portfolio optimization.
# It creates a mapping from stock tickers to their indexes, computes a weighted sum of stock returns (where the weights are decision variables),
# and adds this sum to a linear programming problem, as part of the objective function.

# --- Ensure series are labeled by tickers (robust even if they already are) ---
portfolio_mean_returns_series = pd.Series(portfolio_mean_returns_series, index=symbols)
betastocks = pd.Series(betastocks, index=symbols)

# Consistent ordering
sorted_tickers = sorted(symbols)

# Reindex to that order (avoids any mismatch)
mu = portfolio_mean_returns_series.reindex(sorted_tickers)
beta = betastocks.reindex(sorted_tickers)

# --- Define LP: maximize expected return ---
prob = pl.LpProblem("PortfolioOptimization", pl.LpMaximize)

# Decision variables: weights per ticker (long-only; adjust bounds as needed)
stocks_vars = pl.LpVariable.dicts("w", sorted_tickers, lowBound=0.0, upBound=1.0)

# Objective: sum_i mu_i * w_i
prob += pl.lpSum(mu[t] * stocks_vars[t] for t in sorted_tickers)

# Constraints
# 1) Full investment
prob += pl.lpSum(stocks_vars[t] for t in sorted_tickers) == 1.0

# 2) Target beta
prob += pl.lpSum(beta[t] * stocks_vars[t] for t in sorted_tickers) == float(BetaTarget)


In [ ]:
# An .lp file is created to summarize all the optimization:

prob.writeLP("WeightsPortfolioCAPM.lp")


In [ ]:
# The entire optimization approach is presented:
print(prob)


In [ ]:
# We proceed to solve the optimization:

prob.solve() # This line executes the optimization problem and returns the status code of the solution; 1 indicates that an optimal solution was found successfully; 0 means the solver did not find an optimal solution


In [ ]:
# The status of the optimization is indicated:

print ("Status:", pl.LpStatus[prob.status])


In [ ]:
# The results are printed:

print("\033[1mAsset Allocation (%)\033[0m")
print("________________")
for v in prob.variables():
    print (v.name, "=", round(v.varValue*100,2))
print("\n")

print("\033[1mPortfolio Performance\033[0m")
print("_____________________")
print("Beta =", BetaTarget)
print ("Expected Return (%)=", round(pl.value(prob.objective)*100,2))
CAPMreturnTarget=risk_free_rate+BetaTarget*(market_mean_return-risk_free_rate)
print("Required Return (%)=", round(CAPMreturnTarget*100,2))
